In [1]:
import json 
import requests
from substrateinterface.utils.ss58 import ss58_decode, ss58_encode


polkaholic_url = "https://api.polkaholic.io/search/xcmtransfers"

In [2]:
# acala --> interlay xcm data
r = requests.post(polkaholic_url, json={"chainID": "acala", "chainIDDest": "interlay"})
acala_interlay = r.json()

In [3]:
# moonbeam --> interlay xcm data
r = requests.post(polkaholic_url, json={"chainID": "moonbeam", "chainIDDest": "interlay"})
moonbeam_interlay = r.json()

In [28]:
moonbeam_interlay[0]

{'msgHash': '0xe17f82622410b7ed5669cab0aed4d27cdfeff4779c42e457a366fa20f5afa68e',
 'extrinsicHash': '0x9dcd31cc5eec5337a37998f36a89860e36c94e5764cbf46fdd1fc175b39f3ed7',
 'extrinsicID': '1651428-19',
 'chainID': 2004,
 'chainIDDest': 2032,
 'blockNumber': '1651428',
 'fromAddress': '0x07d6e8987a17b95eee44fbd2b7bb65c34442a5c7',
 'destAddress': '0x5835e670318b79c3c79ebb94b55171d83b031dfa29c761c88550613cb911d33e',
 'sectionMethod': 'xTokens:TransferredMultiAssets',
 'asset': '{"Token":"IBTC"}',
 'rawAsset': '{"Token":"IBTC"}',
 'nativeAssetChain': '{"Token":"IBTC"}~2032',
 'blockNumberDest': 980010,
 'sourceTS': 1660491354,
 'destTS': 1660491380,
 'amountSent': 0.3204402,
 'amountReceived': 0.32043948,
 'status': None,
 'relayChain': 'polkadot',
 'incomplete': 0,
 'amountSentUSD': 0,
 'amountReceivedUSD': 0,
 'chainName': 'Moonbeam',
 'id': 'moonbeam',
 'idDest': 'interlay',
 'chainDestName': 'Interlay'}

In [4]:
acala_startblock = 1638215
interlay_startblock = 975127

ibtc_asset_code = '{"Token":"IBTC"}'

# Total XCM transfers since start of attack

In [5]:
acala_interlay_since_attack = [x for x in acala_interlay if int(x["blockNumberDest"] or 0) >= interlay_startblock and x["asset"] == ibtc_asset_code]

In [6]:
moonbeam_interlay_since_attack = [x for x in moonbeam_interlay if int(x["blockNumberDest"] or 0) >= interlay_startblock and x["asset"] == ibtc_asset_code]

In [7]:
len(acala_interlay_since_attack)

9

In [8]:
len(moonbeam_interlay_since_attack)

90

In [ ]:
moonbeam_temp = [moonbeam_interlay_since_attack]

## iBTC Sent via XCM 

### Acala

In [9]:
iBTC_sent_by_acala = sum([x["amountSent"] for x in acala_interlay_since_attack])
iBTC_sent_by_acala

3.6236798300000004

In [10]:
iBTC_received_from_acala =  sum([x["amountReceived"] for x in acala_interlay_since_attack])
iBTC_received_from_acala

3.6236733500000002

### Moonbeam

In [11]:
iBTC_sent_by_moonbeam = sum([x["amountSent"] for x in moonbeam_interlay_since_attack])
iBTC_sent_by_moonbeam

2.8553268500000004

In [12]:
iBTC_received_from_moonbeam =  sum([x["amountReceived"] for x in moonbeam_interlay_since_attack])
iBTC_received_from_moonbeam

2.8552620500000008

### Accounts on Interlay that received iBTC via XCM after attack started

In [13]:
interlay_prefix = 2032
def ss58convert(pk, prefix): 
    return ss58_encode(pk, ss58_format=prefix)

In [14]:
accounts_from_acala = [(ss58_encode(x["destAddress"],ss58_format=interlay_prefix), x["amountReceived"]) for x in acala_interlay_since_attack]
accounts_from_moonbeam = [(ss58_encode(x["destAddress"],ss58_format=interlay_prefix), x["amountReceived"]) for x in moonbeam_interlay_since_attack]

In [15]:
len(accounts_from_moonbeam)There are a lot

90

In [16]:
accounts_xcm_ibtc = {}

for (acc, amount) in accounts_from_acala:
    if(acc not in accounts_xcm_ibtc):
        accounts_xcm_ibtc[acc] = {"acala" : 0, "moonbeam": 0}
    accounts_xcm_ibtc[acc]["acala"] += amount

for (acc, amount) in accounts_from_moonbeam:
    if(acc not in accounts_xcm_ibtc):
        accounts_xcm_ibtc[acc] = {"acala" : 0, "moonbeam": 0}
    accounts_xcm_ibtc[acc]["moonbeam"] += amount
    
accounts_xcm_ibtc

{'wdBywqU5hGgMxQUUyU8gvmoFK4ubhTH1c2NhxAPKQB9JHVhCB': {'acala': 0.00308344,
  'moonbeam': 0.024617229999999997},
 'wdCDPsf4k56kon1Gyw4RyqCxoL4cNPozasUMmBDnv47g9eJqb': {'acala': 0.14495791000000002,
  'moonbeam': 0.45611322999999987},
 'wdB8KP6AAej73hvBHvw8XDxTTPGMp6eauLxgce1nE96XRTRy5': {'acala': 0.00413217,
  'moonbeam': 0.15706239000000002},
 'wdAE42j6vE7uTzKiFY1tJX4yC2DyUUeTNAkjGh5aTpUzfAw83': {'acala': 3.0062178399999997,
  'moonbeam': 0},
 'wdBg6oWLHw5Xd3WTjP71ZYFn7bQWbNWpEBrb3rJBhwb3AuBcV': {'acala': 0.39915739,
  'moonbeam': 0.62632282},
 'wdBSKkRCKpZrTvpB6jT59BMCSfR983KRemucMACtCwN5P17VK': {'acala': 0.0661246,
  'moonbeam': 0.13856441},
 'wd9VYN3oRkKbPeWxrcwEZH7ko2nGhcjyeA66LzxgNzLUX6qjQ': {'acala': 0,
  'moonbeam': 1.0247127699999998},
 'wdCwFgoKxgrRq1ZpxS93iV2q5e5tmkT3Jw3GAr6kiEezvkbkS': {'acala': 0,
  'moonbeam': 0.09999712000000001},
 'wdAMHAH58VFWYjuCR4kutUoY1gkWZcGq3cSEwjf6jtjW77ZLm': {'acala': 0,
  'moonbeam': 0.01529126},
 'wdAe2cbTgSjLTwgfFw2cF64yG8FaCT1z8taMkphL4WAHSG

# Comparing against iBTC redeem requests since 

Now we check if any iBTC moved over from Acala or Moonbeam has already been redeemed.... this is where the problems could arise

In [17]:
def toBTCDecimals(amount):
    return amount / 100_000_000

In [18]:
graphql_api = "https://api.interlay.io/graphql/graphql"

In [19]:
redeem_requests_since_attack_query = """
query MyQuery {
  redeems(where: {request: {height: {absolute_gt: 975127}}}) {
    id
    status
    userParachainAddress
    request {
      requestedAmountBacking
      height {
        absolute
      }
    }
  }
}
"""

In [20]:
r = requests.post(graphql_api, json={'query': redeem_requests_since_attack_query})
redeem_requests_since_attack = r.json()["data"]["redeems"]

In [21]:
redeem_requests_since_attack

[{'id': '0xfb9ac35558921b1feca273c1b5f4336ec092805700764ad5094767ddc33ee296',
  'status': 'Completed',
  'userParachainAddress': 'wdBg6oWLHw5Xd3WTjP71ZYFn7bQWbNWpEBrb3rJBhwb3AuBcV',
  'request': {'requestedAmountBacking': '14922312',
   'height': {'absolute': 975778}}},
 {'id': '0x2f70855d13a98b8eab32252d4f0da63d7206501b1275c8309ae58671bcba2a2f',
  'status': 'Completed',
  'userParachainAddress': 'wdBg6oWLHw5Xd3WTjP71ZYFn7bQWbNWpEBrb3rJBhwb3AuBcV',
  'request': {'requestedAmountBacking': '24772812',
   'height': {'absolute': 975849}}},
 {'id': '0xda768c592ed88a6277b19e56b24ac3bcd0bf2f6205949379c2f9075b194cca87',
  'status': 'Completed',
  'userParachainAddress': 'wdAE42j6vE7uTzKiFY1tJX4yC2DyUUeTNAkjGh5aTpUzfAw83',
  'request': {'requestedAmountBacking': '299115987',
   'height': {'absolute': 976089}}},
 {'id': '0x4963edd5a3c286b07d6c051bf71841e4be4e77bd1f0435f2cca457f2aa8e253d',
  'status': 'Completed',
  'userParachainAddress': 'wdBiQZXAvkFS3v1653xXZPvKHyHHYzuU3Xt6GRHtDGdc7JDJ9',
  'r

In [22]:
xcm_ibtc_redeems = []

for req in redeem_requests_since_attack:
    if(req["userParachainAddress"] in accounts_xcm_ibtc):
        xcm_ibtc_redeems.append(req)
        
xcm_ibtc_redeems

[{'id': '0xfb9ac35558921b1feca273c1b5f4336ec092805700764ad5094767ddc33ee296',
  'status': 'Completed',
  'userParachainAddress': 'wdBg6oWLHw5Xd3WTjP71ZYFn7bQWbNWpEBrb3rJBhwb3AuBcV',
  'request': {'requestedAmountBacking': '14922312',
   'height': {'absolute': 975778}}},
 {'id': '0x2f70855d13a98b8eab32252d4f0da63d7206501b1275c8309ae58671bcba2a2f',
  'status': 'Completed',
  'userParachainAddress': 'wdBg6oWLHw5Xd3WTjP71ZYFn7bQWbNWpEBrb3rJBhwb3AuBcV',
  'request': {'requestedAmountBacking': '24772812',
   'height': {'absolute': 975849}}},
 {'id': '0xda768c592ed88a6277b19e56b24ac3bcd0bf2f6205949379c2f9075b194cca87',
  'status': 'Completed',
  'userParachainAddress': 'wdAE42j6vE7uTzKiFY1tJX4yC2DyUUeTNAkjGh5aTpUzfAw83',
  'request': {'requestedAmountBacking': '299115987',
   'height': {'absolute': 976089}}},
 {'id': '0x5c8271670e0f222b8fd712d9e3dea7ec485a56264c5056cc9f65593f9c833883',
  'status': 'Completed',
  'userParachainAddress': 'wdCwFgoKxgrRq1ZpxS93iV2q5e5tmkT3Jw3GAr6kiEezvkbkS',
  'r

In [23]:
toBTCDecimals(sum(int(x["request"]["requestedAmountBacking"]) for x in xcm_ibtc_redeems))

3.60102716

# Comparing with attacker accounts

In [24]:


attacker_accounts = [
    "wdCw3nfLyHYP2NcsVgzvXLfTxWU9RKbUjXkPU9jBqpnVWdyAP"
]


In [25]:
attackers_xcm_ibtc = []
for a in attacker_accounts:
    if a in accounts_xcm_ibtc:
        attackers_xcm_ibtc.append(accounts_xcm_ibtc[a])
        
attackers_xcm_ibtc

[]